In [2]:
import requests
from bs4 import BeautifulSoup
import csv


URL = "https://www.tripadvisor.fr/Hotels-g187147-Paris_Ile_de_France-Hotels.html"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(URL, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Trouver les liens des 10 premiers hôtels
hotel_elems = soup.select('.listing_title a')[:40]
hotel_links = ['https://www.tripadvisor.fr' + elem['href'] for elem in hotel_elems]

hotel_data = []

for link in hotel_links:
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupérer le nom de l'hôtel
    hotel_name = soup.select_one('h1.QdLfr.b.d.Pn').text.strip()

    # Récupérer l'adresse
    address = soup.select_one('.fHvkI.PTrfg').text.strip()
    
    # Récupérer Avis
    avis = soup.select_one('.iypZC.Mc._R.b').text.strip()
    
     # Récupérer notes
    notes = soup.select_one('.uwJeR.P')
    note = notes.text.strip() if notes else 'Non disponible'

    equipements_list = soup.select('.OsCbb.K')  # Changez '.equipement-item' par la classe appropriée

    equipements = [item.text.strip() for item in equipements_list]




    # Récupérer le numéro de téléphone 
    phone_elem = soup.select_one('.zNXea.NXOxh.NjUDn')
    phone = phone_elem.text.strip() if phone_elem else 'Non disponible'

    hotel_data.append({
        'Nom': hotel_name,
        'Adresse': address,
        'Téléphone': phone,
        'Avis': avis,
        'Note':note,
        'equipement': equipements
    })

# print données
# for data in hotel_data:
#     print(data)

# Écrire les données dans un fichier CSV
with open('hotel.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Nom', 'Adresse', 'Téléphone', 'Avis', 'Note', 'equipement']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()  # ...en-tête du fichier CSV
    for data in hotel_data:
        # Convertir la liste des équipements en une seule chaîne pour le CSV
        data['equipement'] = ', '.join(data['equipement'])
        writer.writerow(data)
